In [45]:
import pandas as pd
import numpy as np
import ta
import talib
import plotly.graph_objects as go

import datetime

import util_functions as uf

In [142]:
sma_value = [50]
close='Close'
sym='NVDA'

today = datetime.datetime.now()
end_date = datetime.datetime(today.year, today.month, today.day-1)
start_date = datetime.datetime(today.year-1, 1, 1)

In [143]:
df = uf.getdata(sym, sma_value, close, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [144]:
df = df.dropna(subset=['sma_50']).reset_index(drop='TRUE')

In [146]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = -1
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle]['Low'] > df.iloc[i]['Low']:
            pivotLow=0
        if df.iloc[candle]['High'] < df.iloc[i]['High']:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [147]:
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window=3), axis=1)

In [148]:
def pointpos(x):
    if x['isPivot']==-1:
        return x['Low']-3
    elif x['isPivot']==1:
        return x['High']+3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [149]:
def detect_structure(candle, backcandles, window, close, zone_width):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    if (candle <= (backcandles+window)) or (candle+window+1 >= len(df)):
        return 0
    
    localdf = df.iloc[candle-backcandles-window:candle-window] #window must be greater than pivot window to avoid look ahead bias
    highs = localdf[localdf['isPivot'] == 1]['High'].values
    lows = localdf[localdf['isPivot'] == -1]['Low'].values
    levelbreak = 0
    
    if len(lows)>=1:
        support_condition = True
        mean_low = lows.mean()
        for low in lows:
            if abs(low-mean_low)>zone_width:
                support_condition = False
                break
        if support_condition and (mean_low - df.loc[candle][close])>0:
            levelbreak = -1

    if len(highs)>=1:
        resistance_condition = True
        mean_high = highs.mean()
        for high in highs:
            if abs(high-mean_high)>zone_width:
                resistance_condition = False
                break
        if resistance_condition and (df.loc[candle][close]-mean_high)>0:
            levelbreak = 1
    return levelbreak

In [150]:
df['pattern_detected'] = df.apply(lambda row: detect_structure(row.name, backcandles=60, window=6, close='Close', zone_width=10), axis=1)

In [151]:
localdf = df.iloc[39-30-6:39-6]
highs = localdf[localdf['isPivot'] == 1]['High'].values


In [152]:
print(highs)
print(highs.mean())

[275.89001465 280.         280.         277.8999939  281.1000061
 290.57998657]
280.9116668701172


In [153]:
def pointpos_pattern(x):
    if x['pattern_detected']==1:
        return x['High']+3
    elif x['pattern_detected']==-1:
        return x['Low']-3
    else:
        return np.nan
df['pointpos_pattern'] = df.apply(lambda row: pointpos_pattern(row), axis=1)

In [154]:
df

,Date,Open,High,Low,Close,Volume,ticker,sma_50,SMASignal,isPivot,pointpos,pattern_detected,pointpos_pattern
0,2023-03-15,237.610001,242.860001,233.600006,242.279999,52448600,NVDA,204.571799,0,0,NaN,0,NaN
1,2023-03-16,240.270004,255.880005,238.940002,255.410004,58325300,NVDA,206.816999,0,0,NaN,0,NaN
2,2023-03-17,259.820007,263.989990,256.679993,257.250000,84854700,NVDA,209.012199,0,0,NaN,0,NaN
3,2023-03-20,256.149994,260.239990,251.300003,259.000000,43274700,NVDA,211.339199,0,0,NaN,0,NaN
4,2023-03-21,261.799988,263.920013,253.809998,261.989990,54740800,NVDA,213.607199,0,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,2024-03-18,903.880005,924.049988,870.849976,884.549988,66897600,NVDA,708.780999,2,0,NaN,0,NaN
254,2024-03-19,867.000000,905.440002,850.099976,893.979980,67217100,NVDA,716.841199,2,-1,847.099976,0,NaN
255,2024-03-20,897.969971,904.099976,882.229980,903.719971,47906300,NVDA,724.464998,2,0,NaN,0,NaN
256,2024-03-21,923.000000,926.479980,904.049988,914.349976,48037200,NVDA,732.123997,2,0,NaN,0,NaN


In [155]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.add_scatter(x=df.index, y=df['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple", symbol='circle'),
                name="pivot")
fig.add_scatter(x=df[df['pattern_detected']==1].index, y=df[df['pattern_detected']==1]['pointpos_pattern'], mode="markers",
                marker=dict(size=8, color="orange", symbol='arrow-bar-up'),
                name="buy_signal")
fig.add_scatter(x=df[df['pattern_detected']==-1].index, y=df[df['pattern_detected']==-1]['pointpos_pattern'], mode="markers",
                marker=dict(size=8, color="orange", symbol='arrow-bar-down'),
                name="sell_signal")

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()



In [141]:
df[df.index==250]

,Date,Open,High,Low,Close,Volume,ticker,sma_50,SMASignal,isPivot,pointpos,pattern_detected,pointpos_pattern
250,2024-03-13,172.770004,173.190002,170.759995,171.130005,52488700,AAPL,183.771199,1,0,NaN,-1,167.759995
